## The simplest workflow pattern executes steps in a predefined order. Each step's output becomes input for the next step, creating a clear chain of operations. This pattern is ideal for tasks with well-defined sequences, like content generation pipelines or data transformation processes.

In [1]:
import { createOpenAI } from "@ai-sdk/openai";
import { generateText, generateObject } from 'ai'
import { config } from 'dotenv'
import { z } from 'zod'

config()

const API_KEY = Deno.env.get('API_KEY')
console.log(API_KEY)
const BASE_URL = Deno.env.get('BASE_URL')
 
const openai = createOpenAI({
  apiKey: API_KEY,
	baseURL: BASE_URL,
});

async function generateMarketingCopy(input: string) {
	const model = openai('gpt-4o-mini')

	// 使用模型生成营销文案
	const { text: copy } = await generateText({
		model,
		prompt:`
		You are a marketing copywriter.
		You are given a product description and you need to generate a persuasive marketing copy for the product.
		
		Product description: ${input}	
		`,
	})


	// 生成文案质量评估
	const { object: qualityMetrics } = await generateObject({
		model,
		schema: z.object({
			persuasion: z.boolean(),
			clarity: z.number(),
			creativity: z.number(),
		}),
		prompt: `
		Evaluate this marketing copy for:
		- Persuasion: How effectively does the copy persuade the reader to take action(true or false)?
		- Clarity: Is the copy clear and easy to understand(1-10)?
		- Creativity: Is the copy creative and engaging(1-10)?
		
		Marketing copy: ${copy}
		`
	})
	
	
	// 如果文案质量不通过，则重新生成文案
	if (!qualityMetrics.persuasion || qualityMetrics.clarity < 8 || qualityMetrics.creativity < 8) {
		const { text: improvedCopy } = await generateText({
			model,
			prompt: `
			Improve this marketing copy for:
			- ${qualityMetrics.persuasion ? '' : 'Persuasion is not effective'}
			- ${qualityMetrics.clarity < 8 ? `Improve the clarity and make it easier to understand` : ''}
			- ${qualityMetrics.creativity < 8 ? `Make it more creative and engaging` : ''}
				
				Marketing copy: ${copy}
			`
		})

		return {
			copy: improvedCopy,
			qualityMetrics
		}
	}

	return { copy, qualityMetrics }
}


const { copy, qualityMetrics } = await generateMarketingCopy('A product that helps people learn to code')
console.log(copy)
console.log(qualityMetrics)

sk-KPEOX9N9cHJ8KpHnoz9BDn85fcPiGffr9aqTryGMpJOwSYzA
**Ignite Your Potential with CodeMaster!**

Are you ready to embark on an electrifying adventure into the realm of coding? Whether you’re starting from scratch or seeking to refine your abilities, **CodeMaster** is your passport to becoming a digital maestro.

Picture this: designing your own apps, crafting stunning websites, and creating immersive games, all while paving the way to a rewarding career filled with limitless opportunities! With CodeMaster, you’ll begin an exhilarating journey curated by industry veterans. Our dynamic, interactive lessons, real-world coding projects, and vibrant support community will empower you to learn at your own pace, all from the comfort of your home.

**Why Choose CodeMaster?**  
- **Trailblazing Content:** Our captivating lessons are designed to spark your curiosity and keep you excited about your learning journey.  
- **Hands-On Experience:** Tackle real-world projects that simulate the challeng